In [26]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [27]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
        y = torch.FloatTensor([1. if self.y[index] else 0.])
        return x, y

    def __len__(self):
        return self.length

In [28]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [29]:
train_dataset = CustomDataset("./binary_classification_data.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [30]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch + 1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[3.7334e-03, 4.1033e-05, 2.7357e-03]], requires_grad=True), Parameter containing:
tensor([0.0539], requires_grad=True)], Cost : 0.673
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0045, 0.0008, 0.0033]], requires_grad=True), Parameter containing:
tensor([-0.0218], requires_grad=True)], Cost : 0.662
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0046, 0.0008, 0.0030]], requires_grad=True), Parameter containing:
tensor([-0.0966], requires_grad=True)], Cost : 0.661
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0014, 0.0041]], requires_grad=True), Parameter containing:
tensor([-0.1702], requires_grad=True)], Cost : 0.650
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0054, 0.0018, 0.0047]], requires_grad=True), Parameter containing:
tensor([-0.2428], requires_grad=True)], Cost : 0.641
Epoch : 6000, Model : [Parameter containing:
tensor([[0.0054, 0.0022, 0.0041]], requires_grad=True), Paramete

In [31]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [[89, 92, 75], [75, 64, 50], [38, 58, 63], [33, 42, 39], [23, 15, 32]]
    ).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs >= torch.FloatTensor([0.5]).to(device))

---------
tensor([[0.7072],
        [0.6254],
        [0.5759],
        [0.5142],
        [0.4576]])
tensor([[ True],
        [ True],
        [ True],
        [ True],
        [False]])
